In [1]:
import codecs

import os
import sys
module_path = "/home/vika/NER_RNN/targer/src/classes"
if module_path not in sys.path:
    sys.path.append(module_path)


from utils import get_words_num


In [2]:
!python --version

Python 3.6.6


In [1]:
!pip3 install --trusted-host=pypi.python.org --trusted-host=files.pythonhosted.org https://download.pytorch.org/whl/cpu/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl --user 

^C
Operation cancelled by user


In [3]:
import torch 
print(torch.__version__) 

0.4.1


In [4]:
torch.cuda.is_available()

True

в консоли

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85


In [77]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [73]:
!pwd

/home/vika/NER_RNN/targer/data/NER/Asqua_CAM_aspects


In [69]:
!nvidia-smi

Sun Aug 18 08:18:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.87                 Driver Version: 390.87                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 26%   45C    P8    18W / 250W |   4358MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 27%   45C    P8    17W / 250W |    448MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [65]:
def read_data_csv(fn, verbose=True, column_no=-1, splitter = ','):
        word_sequences = list()
        tag_sequences = list()
        with codecs.open(fn, 'r', 'utf-8') as f:
            lines = f.readlines()
        curr_words = list()
        curr_tags = list()
        for k in range(len(lines)):
            #print (lines[k])
            line = lines[k].strip()
            if len(line) == 1 or line.startswith('-DOCSTART-'): # new sentence or new document
                if len(curr_words) > 0:
                    word_sequences.append(curr_words)
                    tag_sequences.append(curr_tags)
                    curr_words = list()
                    curr_tags = list()
                continue
            strings = line.split(',')
            #print (strings)
            word = strings[0]
            #print ("word", word)
            tag = strings[column_no] # be default, we take the last tag
            #print ("word", tag)
            curr_words.append(word)
            curr_tags.append(tag)
            if k == len(lines) - 1:
                word_sequences.append(curr_words)
                tag_sequences.append(curr_tags)
        if verbose:
            print('Loading from %s: %d samples, %d words.' % (fn, len(word_sequences), get_words_num(word_sequences)))
        return word_sequences, tag_sequences

In [66]:
a, b = read_data_csv(fn = "CAM_dev.csv")

Loading from CAM_dev.csv: 350 samples, 10092 words.


In [23]:
/home/vika/NER_RNN/targer/data/NER/Asqua_CAM_aspects

/home/vika/NER_RNN/targer/data/NER/Asqua_CAM_aspects


In [24]:
a, b = read_data(fn = "/home/vika/NER_RNN/targer/data/NER/CoNNL_2003_shared_task/dev.txt")

Loading from /home/vika/NER_RNN/targer/data/NER/CoNNL_2003_shared_task/dev.txt: 3250 samples, 51362 words.


In [1]:
!python --version

Python 3.6.6


In [7]:
import allennlp.commands.elmo

In [1]:
import sys,os,os.path
os.environ['HTTP_PROXY']="http://165.225.66.34:10015" # НЕ РАБОТАЕТ =(
os.environ['HTTPS_PROXY']="http://165.225.66.34:10015"

In [17]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file='/home/vika/NER_RNN/targer/embeddings/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json' 
weight_file='/home/vika/NER_RNN/targer/embeddings/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5'
# Compute two different representation for each token.
# Each representation is a linear weighted combination for the
# 3 layers in ELMo (i.e., charcnn, the outputs of the two BiLSTM))
elmo = Elmo(options_file, weight_file, 1, dropout=0)

# use batch_to_ids to convert sentences to character ids
sentences = [["I", "ate", "an", "apple", "for", "breakfast","."], ["I", "ate", "a", "carrot", "for", "breakfast", "."]]
character_ids = batch_to_ids(sentences)

embeddings = elmo(character_ids)

In [1]:
!nvidia-smi

Wed Sep  4 19:05:18 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.87                 Driver Version: 390.87                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 32%   54C    P2    60W / 250W |   6464MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 40%   63C    P2    80W / 250W |  10714MiB / 11178MiB |     35%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [23]:
len(embeddings['elmo_representations'][0][1][5])

1024

In [6]:
from allennlp.commands.elmo import ElmoEmbedder

print ('qy')
elmo = ElmoEmbedder(
    options_file='/home/vika/NER_RNN/targer/embeddings/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json', 
    weight_file='/home/vika/NER_RNN/targer/embeddings/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5'
)
print ('qy')
tokens = ["I", "ate", "an", "apple", "for", "breakfast"]
print ('qy')
vectors = elmo.embed_sentence(tokens)

assert(len(vectors) == 3) # one for each layer in the ELMo output
assert(len(vectors[0]) == len(tokens)) # the vector elements 

import scipy
vectors2 = elmo.embed_sentence(["I", "ate", "a", "carrot", "for", "breakfast"])
scipy.spatial.distance.cosine(vectors[2][3], vectors2[2][3]) # cosine 


qy
qy
qy


0.44779396057128906

In [8]:
len(vectors)

3

In [4]:
!pip3 install --user --trusted-host=pypi.python.org --trusted-host=pypi.org --trusted-host=files.pythonhosted.org allennlp

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f92997ad748>, 'Connection to pypi.org timed out. (connect timeout=15)')': /simple/allennlp/
  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f92997ad6a0>, 'Connection to pypi.org timed out. (connect timeout=15)')': /simple/allennlp/
  Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<pip._vendor.urllib3.connection.VerifiedHTTPSConnection object at 0x7f92997ad518>, 'Connection to pypi.org timed out. (connect timeout=15)')': /simple/allennlp/
  Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ConnectTimeoutError(<